In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should show RTX 2050

In [ ]:
import joblib
import numpy as np
from transformers import BertTokenizer, BertModel
import torch

# -------------------------------
# 1. Initialize BERT tokenizer & model
# -------------------------------
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()

# -------------------------------
# 2. Define embedding function
# -------------------------------
def bert_embeddings(texts, tokenizer, model, max_length=128):
    """
    Generate CLS embeddings for a list of texts.
    """
    inputs = tokenizer(
        texts,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=max_length
    )
    with torch.no_grad():
        outputs = model(**inputs)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
    return cls_embeddings.numpy()

# -------------------------------
# 3. Load trained LightGBM model
# -------------------------------
try:
    model = joblib.load('lightgbm.pkl')
    print("✅ Loaded LightGBM model successfully.")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit()

# -------------------------------
# 4. New article to classify
# -------------------------------
news = ["""🚨 SHOCKING DISCOVERY: Quantum Shoes Let People FLY Instantly!!! 🚨
BREAKING NEWS ‼️ Residents in Silverford claim they have already TESTED the “Quantum Gravity Shoes”...
(etc.)
"""]

# -------------------------------
# 5. Preprocess & predict
# -------------------------------
try:
    X_new = bert_embeddings(news, tokenizer, bert_model)
except Exception as e:
    print(f"❌ Error generating embeddings: {e}")
    exit()

# Prediction
pred = model.predict(X_new)
proba = model.predict_proba(X_new)[0]

# -------------------------------
# 6. Map numeric → real/fake
# -------------------------------
label_map = {0: "real", 1: "fake"}  # adjust if your training was opposite
predicted_label = label_map[pred[0]]

print("\n--- Prediction Result ---")
print(f"Predicted Label: {predicted_label}")
print(f"Fake Probability: {proba[1]:.4f}")
print(f"Real Probability: {proba[0]:.4f}")
